### TSSOS fails to find traceless disipator which exists

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using Plots

Consider simplest possible Lindblad master equation with just one dissipator:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

and dissipator is traceless:

$
A = \begin{pmatrix} a_1 + i b_1 &   a_2 + i b_2\\  a_3 + i b_3 & - a_1 - i b_1
   \end{pmatrix} $
   
$ \operatorname{tr} A = 0$




### Load exact data

In [2]:
γᵗˣᵗ = "0.079477"

parentdir = pwd()
data_dir = parentdir*"\\DATA\\"
println(data_dir)

ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵗˣᵗ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵗˣᵗ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵗˣᵗ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵗˣᵗ)

@assert tᵍ == tᵉ == tˣ == tʸ 

t = convert(Vector{Float64}, tᵉ)
@assert maximum(diff(t)) ≈ minimum(diff(t)) ≈ t[2]-t[1]
Δt = t[2]-t[1]
t_steps = length(t)

C:\Users\Zakhar\Documents\GitHub\POP_fail\DATA\


1256

### Perform POP SID

In [3]:
#using DynamicPolynomials

@polyvar e[1:2]
@polyvar h[1:2]

Hˢʸᵐᵇₐₙ = [ e[1]   0
            0      0. ]

Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [4]:
@polyvar a[1:3]
@polyvar b[1:3]

Aˢʸᵐᵇₐₙ = [ 0      a[2]
            0      0.  ]

Aˢʸᵐᵇ = [ a[1] + im*b[1]           a[2] + im*b[2]
          a[3] + im*b[3]          -a[1] - im*b[1]   ]

2×2 Matrix{Polynomial{true, Complex{Int64}}}:
 a₁ + (0+1im)b₁  a₂ + (0+1im)b₂
 a₃ + (0+1im)b₃  -a₁ + (0-1im)b₁

In [5]:
objₑₓ = 0
objₑₓₐₙ = 0

for ρ in [ρᵍ, ρᵉ, ρˣ, ρʸ]

    # Convert cut ρ series:
    ρ = convert(Vector{Matrix{ComplexF64}}, ρ)

    objₑₓ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
    
    objₑₓₐₙ += LiPoSID.simpson_obj(ρ, t,  Hˢʸᵐᵇₐₙ, [Aˢʸᵐᵇₐₙ])

end # of files (initial states) loop

objₑₓ 

3.1796954746881942a₁⁴ - 0.013938441211284344a₁³a₂ + 0.013509761228743686a₁³a₃ - 0.013452739566579244a₁³b₂ - 0.018880717674252877a₁³b₃ + 10.940820302933638a₁²a₂² - 27.278937190981075a₁²a₂a₃ + 0.013452739566579244a₁²a₂b₁ - 0.0012575996083348737a₁²a₂b₂ - 0.0037727988250046275a₁²a₂b₃ + 29.16440169903429a₁²a₃² + 0.018880717674252877a₁²a₃b₁ + 0.0037727988250046275a₁²a₃b₂ + 0.0012575996083348737a₁²a₃b₃ + 6.3593909493763885a₁²b₁² - 0.013938441211284344a₁²b₁b₂ + 0.013509761228743686a₁²b₁b₃ + 10.940904373368438a₁²b₂² + 27.279441613589867a₁²b₂b₃ + 29.164485769469103a₁²b₃² - 0.007022805603459752a₁a₂³ + 0.006808465612189406a₁a₂²a₃ + 0.0012575996083348737a₁a₂²b₁ - 0.006047872519830426a₁a₂²b₂ - 0.008761861573667231a₁a₂²b₃ - 0.006915635607824613a₁a₂a₃² - 0.013938441211284344a₁a₂b₁² - 0.00016814086959842539a₁a₂b₁b₂ - 54.558378804570964a₁a₂b₁b₃ - 0.007022805603459752a₁a₂b₂² - 0.006915635607824613a₁a₂b₃² + 0.006701295616554283a₁a₃³ - 0.0012575996083348737a₁a₃²b₁ - 0.0074048670467488364a₁a₃²b₂ - 0.0101188

In [6]:
solₑₓⁿᵉʷ, best_methodₑₓⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓ) 
Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓⁿᵉʷ)
Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓⁿᵉʷ)

************************TSSOS************************
TSSOS is launching...
optimum = 0.0005504791761246807

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a local optimal solution giving an upper bound: 0.01185694330882825 and a relative optimality gap: 0.011306464132703569.
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0005504243887982635
Found a local optimal solution giving an upper bound: 0.011856943308885093 and a relative optimality gap: 0.01130651892008683.
No higher TSSOS hierarchy!


2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-1.5623e-16-8.05328e-17im)  (3.87238e-14-2.52033e-15im)
 (3.05743e-16-3.43372e-18im)  (1.5623e-16+8.05328e-17im)

In [7]:
Aˢⁱᵈₑₓ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-1.5623e-16-8.05328e-17im)  (3.87238e-14-2.52033e-15im)
 (3.05743e-16-3.43372e-18im)  (1.5623e-16+8.05328e-17im)

All the elements of dissipator $A$ are zero!!!

In [8]:
Hˢⁱᵈₑₓ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (25.1261+0.0im)           (7.54484e-5-7.1159e-5im)
 (7.54484e-5+7.1159e-5im)  0.0+0.0im

But we can find non-zero dissipator $A$ if look for 

$
    H = \begin{pmatrix} e_1 & 0 \\ 0 & 0
   \end{pmatrix}
$


$
A = \sqrt{\gamma} \sigma = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$

In [9]:
solₑₓₐₙⁿᵉʷ, best_methodₑₓₐₙⁿᵉʷ = LiPoSID.sos_min_newton(objₑₓₐₙ) 
Hˢⁱᵈₑₓₐₙ = subs(Hˢʸᵐᵇₐₙ, solₑₓₐₙⁿᵉʷ)
Aˢⁱᵈₑₓₐₙ = subs(Aˢʸᵐᵇₐₙ, solₑₓₐₙⁿᵉʷ)

************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068260669271202
Global optimality certified!
No higher TSSOS hierarchy!
************************TSSOS************************
TSSOS is launching...
optimum = 0.0013068175921014087
Found a local optimal solution giving an upper bound: 0.011857047039541158 and a relative optimality gap: 0.01055022944743975.
No higher TSSOS hierarchy!


2×2 Matrix{Term{true, Float64}}:
 0.0  0.281936
 0.0  0.0

In [10]:
Hˢⁱᵈₑₓₐₙ

2×2 Matrix{Term{true, Float64}}:
 25.1261  0.0
 0.0      0.0

In [11]:
Aˢⁱᵈₑₓₐₙ

2×2 Matrix{Term{true, Float64}}:
 0.0  0.281936
 0.0  0.0

Difference is 12 orders of magnitude